<center> <strong> Migration des données <strong> </center>

# Initialisation

In [1]:
# IMPORTS ##############################################

# Pandas Dataframe
import pandas as pd

# Numpy

import numpy as np
# Random
import numpy.random as rd
rd.seed(10)

# Name generator
#!pip install censusname
import censusname

# File operations
import os       

# Networking
import requests

# Progress bars
from tqdm import tqdm  

# Paths & URL ##############################################

url_covid = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
url_decisions = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
url_mobility = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
dirpath = './'
f_covid = 'covid.csv'
f_decisions = 'decision.csv'
f_mobility = 'mobility.csv'

# Fonctions ##############################################

def download_data(url,fpath) : 
    if os.path.exists(os.path.join(dirpath, fpath)):
        print('The file %s already exists.' % os.path.join(dirpath, fpath))

    else:
        r = requests.get(url)
        with open(os.path.join(dirpath, fpath), 'wb') as f:
            f.write(r.content)
        print('Downloaded file %s.' % os.path.join(dirpath, fpath))
        
def write_csv(data,fpath) : 
    if os.path.exists(os.path.join(dirpath, fpath)):
        print('The file %s already exists.' % os.path.join(dirpath, fpath))

    else:
        data.to_csv(fpath, index=False, sep=';')
        print('The file %s is saved.' % os.path.join(dirpath, fpath))
        
        
def hospital_name_pattern(name) :
    patterns = [ 
        f'Centre {name}',
        f'Centre de Recherche {name}',
        f'Centre Hospitalier {name}',
        f'Centre Hospitalier Universitaire {name}',
        f'Centre Médical {name}',
        f'Clinique {name}',
        f'Hôpital {name}',
        f'Institut {name}',
        f'Institut de Santé {name}' 
    ]
    return patterns

def random_hospital() :
    name = censusname.generate()
    i = (round(rd.rand()*10))%len(hospital_name_pattern(name))
    return hospital_name_pattern(name)[i]

def hospital_gen() :
    for i in tqdm(df_tmp.index) :
        iso = df_tmp.iso_code.iloc[i]
        pop = df_tmp.population.iloc[i]
        beds = df_tmp.hospital_beds_per_thousand.iloc[i]
        beds = pop*beds/1000
        
        while(beds>0) :
            x = round(rd.uniform(500, np.minimum(10000,beds), 1)[0])
            beds = beds-x
            df_hosp['nom'].append(random_hospital())
            df_hosp['iso'].append(iso)
            df_hosp['lits'].append(x)
    return

def deaths_birth_gen() :
    return round(rd.uniform(1000,3000))

def population_gen() :
    for i in tqdm(df_iso_date.index) :
        iso = df_iso_date.iso_code.iloc[i]
        date = df_iso_date.date.iloc[i]
        deaths = deaths_birth_gen()
        births = deaths_birth_gen()+1500
        
        df_pop['iso'].append(iso)
        df_pop['date'].append(date)
        df_pop['deces'].append(deaths)
        df_pop['naissances'].append(births)
    return

# Téléchargement des données

In [6]:
# Our World In Data
download_data(url_covid,f_covid)
# BSX Oxford
download_data(url_decisions,f_decisions)
# Google Mobility
download_data(url_mobility,f_mobility)

Downloaded file ./covid.csv.
Downloaded file ./decision.csv.
Downloaded file ./mobility.csv.


# Retypage des données

In [2]:
# Our World In Data
covid_dtypes = {
    'iso_code' : 'category', 
    'continent' : 'category', 
    'location' : 'category', 
    'surface' : 'Int64',
    'population' : 'Int64',
    'new_cases': 'Int64',
    'new_deaths': 'Int64',
    'icu_patients': 'Int64',
    'hosp_patients': 'Int64',
    'new_tests': 'Int64',
    'new_vaccinations': 'Int64',
}

# BSX Oxford
decisions_dtypes = {
    'CountryCode' : 'category', 
    'CountryCode' : 'category', 
    'CountryCode' : 'category', 
    'C1_School closing' : 'Int64',
    'C2_Workplace closing' : 'Int64',
    'C3_Cancel public events' : 'Int64',
    'C4_Restrictions on gatherings' : 'Int64', 
    'C5_Close public transport' : 'Int64',
    'C6_Stay at home requirements' : 'Int64',
    'C7_Restrictions on internal movement' : 'Int64',
    'C8_International travel controls' : 'Int64', 
    'E1_Income support' : 'Int64',
    'E2_Debt/contract relief': 'Int64',
    'H1_Public information campaigns' : 'Int64',
    'H2_Testing policy' : 'Int64', 
    'H3_Contact tracing' : 'Int64',
    'H6_Facial Coverings' : 'Int64', 
    'H7_Vaccination policy' : 'Int64',
    'H8_Protection of elderly people' : 'Int64'
}


# Google Mobility
mobility_dtypes = {
    'country_region_code' : 'category', 
    'country_region' : 'category',
    'sub_region_1' : 'category',
    'sub_region_2' : 'category'
}

# Lecture des données

In [3]:
# Our World In Data
df_covid = pd.read_csv(
    # Filename
    f_covid,
    # Column separator
    #sep=',',
    # Decimal separator
    decimal='.',
    # Range of the columns to keep 
    usecols=['iso_code', 
             'continent', 
             'location', 
             'date', 
             'new_cases', 
             'new_deaths', 
             'reproduction_rate', 
             'icu_patients',
             'hosp_patients',
             'new_tests', 
             'new_vaccinations',
             'population', 
             'population_density', 
             'median_age', 
             'aged_65_older',
             'aged_70_older', 
             'gdp_per_capita', 
             'extreme_poverty',
             'cardiovasc_death_rate', 
             'diabetes_prevalence', 
             'female_smokers',
             'male_smokers', 
             'handwashing_facilities', 
             'hospital_beds_per_thousand',
             'life_expectancy', 
             'human_development_index'],
    # Which columns should be parsed as dates
    parse_dates=['date'],
    # Specify some dtypes
    dtype=covid_dtypes
)

# BSX Oxford
df_decisions = pd.read_csv(
    # Filename
    f_decisions,
    # Column separator
    #sep=';',
    # Decimal separator
    decimal='.',
    # Range of the columns to keep 
    usecols=[ 'CountryCode', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
        'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls', 'E1_Income support',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H7_Vaccination policy',
       'H8_Protection of elderly people'],
    # Which columns should be parsed as dates
    parse_dates=['Date'],
    # Specify some dtypes
    dtype=decisions_dtypes
)


# Google Mobility
df_mobility = pd.read_csv(
    # Filename
    f_mobility,
    # Column separator
    #sep=';',
    # Decimal separator
    decimal='.',
    # Range of the columns to keep 
    usecols=[ 
        'country_region_code', 
        'country_region', 
        'sub_region_1',
        'sub_region_2', 
        'date',
        'retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline'],
    # Which columns should be parsed as dates
    parse_dates=['date'],
    # Specify some dtypes
    dtype=mobility_dtypes
)

# Transformation des colonnes

In [4]:
# Our World In Data ##################################

# Code continent
df_covid['code_continent'] = df_covid.continent.cat\
    .rename_categories( { 'Africa' : 'AF' ,
                          'Asia' : 'AS',
                          'Europe' : 'EU',
                          'North America' : 'NA',
                          'Oceania' : 'OC',
                          'South America' : 'SA'
                        } )
# Surface
df_covid['surface'] = df_covid.population / df_covid.population_density
df_covid['surface'] = ((df_covid['surface']*100)//1/100)
# Fumeurs
df_covid['smokers'] = round(( df_covid.female_smokers + df_covid.male_smokers )/2,2)
# Suppression des données qui ne sont pas par pays
df_covid = df_covid.loc[ ~df_covid['iso_code'].str.startswith('OWID') ]
# Corriger les données erronées (e.g. new_tests < 0 )
df_covid.reproduction_rate = abs(df_covid.reproduction_rate)
df_covid.new_cases = abs(df_covid.new_cases)
df_covid.new_deaths = abs(df_covid.new_deaths)
df_covid.hosp_patients = abs(df_covid.hosp_patients)
df_covid.icu_patients = abs(df_covid.icu_patients)
df_covid.new_tests = abs(df_covid.new_tests)



# BSX Oxford ##################################

# On s'accorde sur une même date de début
df_decisions = df_decisions.loc[ df_decisions['Date']>='2020-02-24' ]
# Corriger les données erronées (e.g. plusieurs données le même jour pour le même pays)
df_decisions = df_decisions.groupby(['CountryCode','Date']).mean().reset_index()
df_decisions['C1_School closing'] = df_decisions['C1_School closing'].astype('Int64')
df_decisions['C2_Workplace closing'] = df_decisions['C2_Workplace closing'].astype('Int64')
df_decisions['C3_Cancel public events'] = df_decisions['C3_Cancel public events'].astype('Int64')
df_decisions['C4_Restrictions on gatherings'] = df_decisions['C4_Restrictions on gatherings'].astype('Int64')
df_decisions['C5_Close public transport'] = df_decisions['C5_Close public transport'].astype('Int64')
df_decisions['C6_Stay at home requirements'] = df_decisions['C6_Stay at home requirements'].astype('Int64')
df_decisions['E1_Income support'] = df_decisions['E1_Income support'].astype('Int64')
df_decisions['E2_Debt/contract relief'] = df_decisions['E2_Debt/contract relief'].astype('Int64')
df_decisions['C7_Restrictions on internal movement'] = df_decisions['C7_Restrictions on internal movement'].astype('Int64')
df_decisions['C8_International travel controls'] = df_decisions['C8_International travel controls'].astype('Int64')
df_decisions['H1_Public information campaigns'] = df_decisions['H1_Public information campaigns'].astype('Int64')
df_decisions['H2_Testing policy'] = df_decisions['H2_Testing policy'].astype('Int64')
df_decisions['H3_Contact tracing'] = df_decisions['H3_Contact tracing'].astype('Int64')
df_decisions['H6_Facial Coverings'] = df_decisions['H6_Facial Coverings'].astype('Int64')
df_decisions['H7_Vaccination policy'] = df_decisions['H7_Vaccination policy'].astype('Int64')
df_decisions['H8_Protection of elderly people'] = df_decisions['H8_Protection of elderly people'].astype('Int64')
# Suppression des données qui ne sont pas par pays
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='GUM' ]
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='KIR' ]
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='PRI' ]
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='RKS' ]
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='TKM' ]
df_decisions = df_decisions.loc[ df_decisions['CountryCode']!='VIR' ]




# Google mobility ##################################

# On s'accorde sur une même date de début
df_mobility = df_mobility.loc[ df_mobility['date']>='2020-02-24' ]
# Suppression des données qui ne sont pas par pays
df_mobility = df_mobility\
    .loc[ df_mobility.sub_region_2.isna() ]\
    .loc[ df_mobility.sub_region_1.isna() ]
# Convertion ISO-alpha2 en ISO-alpha3
df_mobility['country_region_code'] = df_mobility.country_region_code.cat\
    .rename_categories( { 
    'AE' : 'ARE' ,
    'AF' : 'AFG',
    'AG' : 'ATG',
    'AO' : 'AGO' ,
    'AR' : 'ARG',
    'AS' : 'ASM' ,
    'AT' : 'AUT',
    'AU' : 'AUS' ,
    'AW' : 'ABW',
    'BA' : 'BIH',
    'BB' : 'BRB' ,
    'BD' : 'BGD' ,
    'BE' : 'BEL' ,
    'BF' : 'BFA' ,
    'BG' : 'BGR' ,
    'BH' : 'BHR' ,
    'BJ' : 'BEN' ,
    'BO' : 'BOL' ,
    'BR' : 'BRA' ,
    'BS' : 'BHS' ,
    'BW' : 'BWA' ,
    'BY' : 'BLR' ,
    'BZ' : 'BLZ' ,
    'CA' : 'CAN' ,
    'CH' : 'CHE' ,
    'CI' : 'CIV' ,
    'CL' : 'CHL' ,
    'CM' : 'CMR' ,
    'CO' : 'COL' ,
    'CR' : 'CRI' ,
    'CV' : 'CPV' ,
    'CZ' : 'CZE' ,
    'DE' : 'DEU' ,
    'DK' : 'DNK' ,
    'DO' : 'DOM' ,
    'EC' : 'ECU' ,
    'EE' : 'EST' ,
    'EG' : 'EGY' ,
    'ES' : 'ESP' ,
    'FI' : 'FIN' ,
    'FJ' : 'FJI' ,
    'FR' : 'FRA' ,
    'GA' : 'GAB' ,
    'GB' : 'GBR' ,
    'GE' : 'GEO' ,
    'GH' : 'GHA' ,
    'GR' : 'GRC' ,
    'GT' : 'GTM' ,
    'GW' : 'GNB' ,
    'HK' : 'HKG' ,
    'HN' : 'HND' ,
    'HR' : 'HRV' ,
    'HT' : 'HTI' ,
    'HU' : 'HUN' ,
    'ID' : 'IDN' ,
    'IE' : 'IRL' ,
    'IL' : 'ISR' ,
    'IN' : 'IND' ,
    'IQ' : 'IRQ' ,
    'IT' : 'ITA' ,
    'JM' : 'JAM' ,
    'JO' : 'JOR' ,
    'JP' : 'JPN' ,
    'KE' : 'KEN' ,
    'KG' : 'KGZ' ,
    'KH' : 'KHM' ,
    'KR' : 'KOR' ,
    'KW' : 'KWT' ,
    'KZ' : 'KAZ' ,
    'LA' : 'LAO' ,
    'LB' : 'LBN' ,
    'LI' : 'LIE' ,
    'LK' : 'LKA' ,
    'LT' : 'LTU' ,
    'LU' : 'LUX' ,
    'LV' : 'LVA' ,
    'LY' : 'LBY' ,
    'MA' : 'MAR' ,
    'MD' : 'MDA' ,
    'MK' : 'MKD' ,
    'ML' : 'MLI' ,
    'MM' : 'MMR' ,
    'MN' : 'MNG' ,
    'MT' : 'MLT' ,
    'MU' : 'MUS' ,
    'MX' : 'MEX' ,
    'MY' : 'MYS' ,
    'MZ' : 'MOZ' ,
    'NE' : 'NER' ,
    'NG' : 'NGA' ,
    'NI' : 'NIC' ,
    'NL' : 'NLD' ,
    'NO' : 'NOR' ,
    'NP' : 'NPL' ,
    'NZ' : 'NZL' ,
    'OM' : 'OMN' ,
    'PA' : 'PAN' ,
    'PE' : 'PER' ,
    'PG' : 'PNG' ,
    'PH' : 'PHL' ,
    'PK' : 'PAK' ,
    'PL' : 'POL' ,
    'PR' : 'PRI' ,
    'PT' : 'PRT' ,
    'PY' : 'PRY' ,
    'QA' : 'QAT' ,
    'RE' : 'REU' ,
    'RO' : 'ROU' ,
    'RS' : 'SRB' ,
    'RU' : 'RUS' ,
    'RW' : 'RWA' ,
    'SA' : 'SAU' ,
    'SE' : 'SWE' ,
    'SG' : 'SGP' ,
    'SI' : 'SVN' ,
    'SK' : 'SVK' ,
    'SN' : 'SEN' ,
    'SV' : 'SLV' ,
    'TG' : 'TGO' ,
    'TH' : 'THA' ,
    'TJ' : 'TJK' ,
    'TR' : 'TUR' ,
    'TT' : 'TTO' ,
    'TW' : 'TWN' ,
    'TZ' : 'TZA' ,
    'UA' : 'UKR' ,
    'UG' : 'UGA' ,
    'US' : 'USA' ,
    'UY' : 'URY' ,
    'VE' : 'VEN' ,
    'VN' : 'VNM' ,
    'YE' : 'YEM' ,
    'ZA' : 'ZAF' ,
    'ZM' : 'ZMB' ,
    'ZW' : 'ZWE' } )
# Corriger les données erronnées (e.g. plusieurs données le même jour pour le même pays)
df_mobility = df_mobility.loc[ : , 
                            ['country_region_code', 'date',
                               'retail_and_recreation_percent_change_from_baseline',
                               'grocery_and_pharmacy_percent_change_from_baseline',
                               'parks_percent_change_from_baseline',
                               'transit_stations_percent_change_from_baseline',
                               'workplaces_percent_change_from_baseline',
                               'residential_percent_change_from_baseline'
                              ] ].drop_duplicates()
df_mobility = df_mobility.groupby(['country_region_code','date']).mean().reset_index()
df_mobility.retail_and_recreation_percent_change_from_baseline = round(df_mobility.retail_and_recreation_percent_change_from_baseline,2)
df_mobility.grocery_and_pharmacy_percent_change_from_baseline = round(df_mobility.grocery_and_pharmacy_percent_change_from_baseline,2)
df_mobility.parks_percent_change_from_baseline = round(df_mobility.parks_percent_change_from_baseline,2)
df_mobility.transit_stations_percent_change_from_baseline = round(df_mobility.transit_stations_percent_change_from_baseline,2)
df_mobility.workplaces_percent_change_from_baseline = round(df_mobility.workplaces_percent_change_from_baseline,2)
df_mobility.residential_percent_change_from_baseline = round(df_mobility.residential_percent_change_from_baseline,2)
df_mobility = df_mobility.loc[ df_mobility['country_region_code']!='ASM' ]
df_mobility = df_mobility.loc[ df_mobility['country_region_code']!='PRI' ]
df_mobility = df_mobility.loc[ df_mobility['country_region_code']!='REU' ]
#df_covid.info()
#df_decisions.info()
#df_mobility.sample()

# Tables finales

In [5]:
pays = df_covid.loc[ : , ['iso_code',
                               'location',
                               'code_continent',
                               'population',
                               'surface',
                               'median_age',
                               'aged_65_older',
                               'aged_70_older',
                               'gdp_per_capita',
                              'extreme_poverty',
                              'diabetes_prevalence',
                              'smokers',
                              'handwashing_facilities',
                              'hospital_beds_per_thousand',
                              'life_expectancy',
                              'human_development_index',
                              'cardiovasc_death_rate' ] ].drop_duplicates()

continent = df_covid.loc[ : , ['code_continent','continent']].drop_duplicates()

stat_covid = df_covid.loc[ : , ['date', 
                                'iso_code', 
                                'new_cases', 
                                'new_deaths', 
                                'reproduction_rate', 
                                'icu_patients', 
                                'hosp_patients',
                                'new_tests',
                                'new_vaccinations'
                               ] ].drop_duplicates()

distanciation = df_decisions.loc[ : , 
                                 ['CountryCode', 'Date',
                               'C1_School closing',
                               'C2_Workplace closing',
                               'C3_Cancel public events',
                               'C4_Restrictions on gatherings',
                               'C5_Close public transport',
                               'C6_Stay at home requirements',
                               'C7_Restrictions on internal movement',
                               'C8_International travel controls'
                              ] ].drop_duplicates()

sanitaire = df_decisions.loc[ : , 
                                 ['CountryCode', 'Date',
                               'H1_Public information campaigns',
                               'H2_Testing policy',
                               'H3_Contact tracing',
                               'H6_Facial Coverings',
                               'H7_Vaccination policy',
                               'H8_Protection of elderly people'
                              ] ].drop_duplicates()

economique = df_decisions.loc[ : , 
                                 ['CountryCode', 'Date',
                               'E1_Income support',
                               'E2_Debt/contract relief',
                               'E3_Fiscal measures',
                               'E4_International support',
                               'H4_Emergency investment in healthcare',
                               'H5_Investment in vaccines'
                              ] ].drop_duplicates()

lieux = df_mobility

# TESTS ###############################################################

#economique.loc[ economique['E4_International support'] > 0].tail(10)
#sanitaire.sample(1)
#distanciation.sample(1)
#stat_covid.loc[ stat_covid.iso_code == 'FRA'].tail(10)
#pays.sample()
#continent.sample(1)
#lieux.loc[ lieux.country_region_code == 'EGY'].loc[ lieux.date == '2020-03-16']
#lieux.loc[ lieux['country_region_code']=='PRI' ]
#lieux.sample()

In [6]:
payscsv = "pays.csv"
write_csv(pays,payscsv)

contcsv = "continent.csv"
write_csv(continent,contcsv)

statcovidcsv = "stat_covid.csv"
write_csv(stat_covid,statcovidcsv)

distanciationcsv = "distanciation.csv"
write_csv(distanciation,distanciationcsv)

sanitairecsv = "sanitaire.csv"
write_csv(sanitaire,sanitairecsv)

ecocsv = "eco.csv"
write_csv(economique,ecocsv)

statlieuxcsv = "stat_lieux.csv"
write_csv(lieux,statlieuxcsv)

The file ./pays.csv is saved.
The file ./continent.csv is saved.
The file ./stat_covid.csv is saved.
The file ./distanciation.csv is saved.
The file ./sanitaire.csv is saved.
The file ./eco.csv is saved.
The file ./stat_lieux.csv is saved.


# Générer des données pour la table Population

In [6]:
df_iso_date = df_covid.loc[ : , ['iso_code','date']].drop_duplicates().reset_index()

df_pop = { 
    'iso' : [] , 
    'date' : [],
    'deces' : [],
    'naissances' : []
}

population_gen()

df_pop = pd.DataFrame.from_dict(df_pop)
popcsv = "population.csv"
write_csv(df_pop,popcsv)

100%|██████████| 82569/82569 [00:06<00:00, 12212.10it/s]


The file ./population.csv is saved.


# Générer les données des hôpitaux 

<strong> Remarque : </strong> <br>
Les données sont inventées (liste des hôpitaux par pays, nombre de lits, ... etc).
Mais elles sont cohérentes avec le vrai nombre de lits d'hôpitaux par pays.

In [9]:
df_hosp = { 
    'nom' : [] , 
    'iso' : [],
    'lits' : []
}

df_tmp = df_covid\
    .loc[ :, ['iso_code','population','hospital_beds_per_thousand'] ]\
    .drop_duplicates()\
    .reset_index()\
    .loc[ :, ['iso_code','population','hospital_beds_per_thousand'] ]

# /!\ Generer les données peut prendre du temps  /!\ 
# --------------------------------------------------

hospital_gen() 
df_hosp = pd.DataFrame.from_dict(df_hosp)
hospcsv = "hopitaux.csv"
write_csv(df_hosp,hospcsv)

100%|██████████| 208/208 [06:46<00:00,  1.95s/it] 

The file ./hopitaux.csv is saved.
